# Парсинг с сайта https://allpainters.ru/: название художника, названия картины и изображение

In [ ]:
# Импорт необходимых библиотек

import csv
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
# Устанавливаем ссылку на страницу, с которой будем парсить данные
base_url = 'https://allpainters.ru/mikelandzhelo-01.html' #Поменять ссылку

# Устанавливаем папку, в которую будут сохраняться изображения
save_folder = '/content/drive/MyDrive/парсинг художников/Микеланджело' #Поменять папку
os.makedirs(save_folder, exist_ok=True)

# Получаем HTML-код страницы
response = requests.get(base_url)
html = response.text

# Создаем объект BeautifulSoup для работы с HTML-кодом
soup = BeautifulSoup(html, 'html.parser')

# Находим все блоки с тегом div и классом tag_block
tag_blocks = soup.find_all('div', class_='tag_block')

In [ ]:
# Создаем списки для хранения заголовков, ссылок на изображения и имен файлов изображений
titles = []
image_urls = []
image_filenames = []

# Проходимся по каждому блоку tag_block и получаем заголовок и ссылку на страницу с изображением
for tag_block in tag_blocks:
    title = tag_block.find('img')['title']
    titles.append(title)
    image_page_url = tag_block.find('a')['href']
    
    # Получаем полную ссылку на страницу с изображением и получаем HTML-код этой страницы
    full_image_page_url = urljoin(base_url, image_page_url)
    image_page_response = requests.get(full_image_page_url)
    image_page_html = image_page_response.text
    
    # Создаем объект BeautifulSoup для работы с HTML-кодом страницы с изображением
    image_page_soup = BeautifulSoup(image_page_html, 'html.parser')
    
    # Находим тег img с классом alignnone и получаем ссылку на изображение
    image_element = image_page_soup.find('img', class_='alignnone')
    image_url = urljoin('https://allpainters.ru', image_element['src'])
    image_urls.append(image_url)

    # Получаем имя файла изображения и сохраняем его в указанную папку
    image_filename = os.path.basename(image_url)
    save_path = os.path.join(save_folder, image_filename)
    urllib.request.urlretrieve(image_url, save_path)

    # Добавляем имя файла изображения в список
    image_filenames.append(image_filename)

In [ ]:
# Создаем CSV-файл и записываем в него данные

csv_file = '/content/drive/MyDrive/парсинг художников/Микеланджело.csv' # Указываем имя CSV-файла
with open(csv_file, 'w', newline='', encoding='utf-8') as file: # Открываем файл для записи
    writer = csv.writer(file) # Создаем объект writer для записи данных в CSV-файл
    writer.writerow(['Название', 'Картинка', 'Название файла']) # Записываем заголовки столбцов
    for i in range(len(titles)):
        writer.writerow([titles[i], image_urls[i], image_filenames[i]]) # Записываем данные в CSV-файл